![pageindex_banner](https://pageindex.ai/static/images/pageindex_banner.jpg)

<p align="center"><i>Reasoning-based RAG&nbsp; ✧ &nbsp;No Vector DB&nbsp; ✧ &nbsp;No Chunking&nbsp; ✧ &nbsp;Human-like Retrieval</i></p>

<p align="center">
  <a href="https://vectify.ai">🏠 Homepage</a>&nbsp; • &nbsp;
  <a href="https://dash.pageindex.ai">🖥️ Dashboard</a>&nbsp; • &nbsp;
  <a href="https://docs.pageindex.ai/quickstart">📚 API Docs</a>&nbsp; • &nbsp;
  <a href="https://github.com/VectifyAI/PageIndex">📦 GitHub</a>&nbsp; • &nbsp;
  <a href="https://discord.com/invite/VuXuf29EUj">💬 Discord</a>&nbsp; • &nbsp;
  <a href="https://ii2abc2jejf.typeform.com/to/tK3AXl8T">✉️ Contact</a>&nbsp;
</p>

<div align="center">

[![Star us on GitHub](https://img.shields.io/github/stars/VectifyAI/PageIndex?style=for-the-badge&logo=github&label=⭐️%20Star%20Us)](https://github.com/VectifyAI/PageIndex) &nbsp;&nbsp; [![Follow us on X](https://img.shields.io/badge/Follow%20Us-000000?style=for-the-badge&logo=x&logoColor=white)](https://twitter.com/VectifyAI)

</div>

---

# Simple Vectorless RAG with PageIndex

## PageIndex Introduction
PageIndex is a new **reasoning-based**, **vectorless RAG** framework that performs retrieval in two steps:  
1. Generate a tree structure index of documents  
2. Perform reasoning-based retrieval through tree search  

<div align="center">
  <img src="https://docs.pageindex.ai/images/cookbook/vectorless-rag.png" width="70%">
</div>

Compared to traditional vector-based RAG, PageIndex features:
- **No Vectors Needed**: Uses document structure and LLM reasoning for retrieval.
- **No Chunking Needed**: Documents are organized into natural sections rather than artificial chunks.
- **Human-like Retrieval**: Simulates how human experts navigate and extract knowledge from complex documents. 
- **Transparent Retrieval Process**: Retrieval based on reasoning — say goodbye to approximate semantic search ("vibe retrieval").

## 📝 Notebook Overview

This notebook demonstrates a simple, minimal example of **vectorless RAG** with PageIndex. You will learn how to:
- [x] Build a PageIndex tree structure of a document
- [x] Perform reasoning-based retrieval with tree search
- [x] Generate answers based on the retrieved context

> ⚡ Note: This is a **minimal example** to illustrate PageIndex's core philosophy and idea, not its full capabilities. More advanced examples are coming soon.

---

## Step 0: Preparation



#### 0.1 Install PageIndex

In [1]:
%pip install -q --upgrade pageindex

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 0.2 Setup PageIndex

In [2]:
from pageindex import PageIndexClient
import pageindex.utils as utils
import os
from dotenv import load_dotenv

load_dotenv()

# Get your PageIndex API key from https://dash.pageindex.ai/api-keys
PAGEINDEX_API_KEY = os.getenv("PAGEINDEX_API_KEY")
if not PAGEINDEX_API_KEY:
        raise ValueError("PAGEINDEX_API_KEY が .env ファイルに設定されていません。")
pi_client = PageIndexClient(api_key=PAGEINDEX_API_KEY)

#### 0.3 Setup LLM

Choose your preferred LLM for reasoning-based retrieval. In this example, we use OpenAI’s GPT-4.1.

In [3]:
from openai import AsyncAzureOpenAI 
import os

AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT") 

if not all([AZURE_OPENAI_KEY, AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_API_VERSION, AZURE_OPENAI_DEPLOYMENT]):
    raise ValueError("Azure OpenAIの環境変数が.envファイルに正しく設定されていません。(AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_API_VERSION, AZURE_OPENAI_DEPLOYMENT)")

async def call_llm(prompt, temperature=0):
    client = AsyncAzureOpenAI(
        api_key=AZURE_OPENAI_KEY,
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_version=AZURE_OPENAI_API_VERSION,
    )
    response = await client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT, 
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content.strip()

## Step 1: PageIndex Tree Generation

#### 1.1 Submit a document for generating PageIndex tree

In [4]:
import os 
# requests はダウンロード不要なため削除

# 送信するローカルファイルのパスを指定
pdf_path = "merged_document.pdf"

# ファイルが本当に存在するかを確認
if os.path.exists(pdf_path):
    print(f"ローカルファイル '{pdf_path}' を送信します...")
    
    # PageIndexクライアントに 'merged_document.pdf' を送信
    doc_id = pi_client.submit_document(pdf_path)["doc_id"]
    print('Document Submitted:', doc_id)
else:
    print(f"エラー: ファイル '{pdf_path}' が見つかりません。")
    print("スクリプトと同じディレクトリに 'merged_document.pdf' があるか確認してください。")

ローカルファイル 'merged_document.pdf' を送信します...
Document Submitted: pi-cmh3i108z04o50cr1zg9ytcci


#### 1.2 Get the generated PageIndex tree structure

In [6]:
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('Simplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

Simplified Tree Structure of the Document:
[{'title': '第1章 当座預金',
  'node_id': '0000',
  'prefix_summary': '# 第1章 当座預金\n',
  'nodes': [{'title': '第1節 基本業務',
             'node_id': '0001',
             'prefix_summary': '## 第1節 基本業務\n',
             'nodes': [{'title': '第1項 一般事項',
                        'node_id': '0002',
                        'summary': 'This document outlines the general matte...'},
                       {'title': '第2項 口座開設申込者の信用調査',
                        'node_id': '0003',
                        'prefix_summary': 'This section details the credit investig...',
                        'nodes': [{'title': '1. 申込書の取入れ',
                                   'node_id': '0004',
                                   'summary': 'This document outlines the procedure for...'},
                                  {'title': '2. 実態の確認',
                                   'node_id': '0005',
                                   'summary': '#### 2. 実態の確認\n\n(1) 申込者からの聴取、関係書類の取入などにより..

## Step 2: Reasoning-Based Retrieval with Tree Search

#### 2.1 Use LLM for tree search and identify nodes that might contain relevant context

In [7]:
import json

query = "What are the conclusions in this document?"

tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

tree_search_result = await call_llm(search_prompt)

#### 2.2 Print retrieved nodes and reasoning process

In [8]:
node_map = utils.create_node_mapping(tree)
tree_search_result_json = json.loads(tree_search_result)

print('Reasoning Process:')
utils.print_wrapped(tree_search_result_json['thinking'])

print('\nRetrieved Nodes:')
for node_id in tree_search_result_json["node_list"]:
    node = node_map[node_id]
    print(f"Node ID: {node['node_id']}\t Page: {node['page_index']}\t Title: {node['title']}")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Step 3: Answer Generation

#### 3.1 Extract relevant context from retrieved nodes

In [58]:
node_list = json.loads(tree_search_result)["node_list"]
relevant_content = "\n\n".join(node_map[node_id]["text"] for node_id in node_list)

print('Retrieved Context:\n')
utils.print_wrapped(relevant_content[:1000] + '...')

Retrieved Context:

## 5. Conclusion, Limitations, and Future Work

In this work, we share our journey in enhancing model reasoning abilities through reinforcement
learning. DeepSeek-R1-Zero represents a pure RL approach without relying on cold-start data,
achieving strong performance across various tasks. DeepSeek-R1 is more powerful, leveraging cold-
start data alongside iterative RL fine-tuning. Ultimately, DeepSeek-R1 achieves performance
comparable to OpenAI-o1-1217 on a range of tasks.

We further explore distillation the reasoning capability to small dense models. We use DeepSeek-R1
as the teacher model to generate 800K training samples, and fine-tune several small dense models.
The results are promising: DeepSeek-R1-Distill-Qwen-1.5B outperforms GPT-4o and Claude-3.5-Sonnet on
math benchmarks with $28.9 \%$ on AIME and $83.9 \%$ on MATH. Other dense models also achieve
impressive results, significantly outperforming other instructiontuned models based on the same
underlying che

#### 3.2 Generate answer based on retrieved context

In [59]:
answer_prompt = f"""
Answer the question based on the context:

Question: {query}
Context: {relevant_content}

Provide a clear, concise answer based only on the context provided.
"""

print('Generated Answer:\n')
answer = await call_llm(answer_prompt)
utils.print_wrapped(answer)

Generated Answer:

The conclusions in this document are:

- DeepSeek-R1-Zero, a pure reinforcement learning (RL) approach without cold-start data, achieves
strong performance across various tasks.
- DeepSeek-R1, which combines cold-start data with iterative RL fine-tuning, is more powerful and
achieves performance comparable to OpenAI-o1-1217 on a range of tasks.
- Distilling DeepSeek-R1’s reasoning capabilities into smaller dense models is promising; for
example, DeepSeek-R1-Distill-Qwen-1.5B outperforms GPT-4o and Claude-3.5-Sonnet on math benchmarks,
and other dense models also show significant improvements over similar instruction-tuned models.

These results demonstrate the effectiveness of the RL-based approach and the potential for
distilling reasoning abilities into smaller models.


---

## 🎯 What's Next

This notebook has demonstrated a **basic**, **minimal** example of **reasoning-based**, **vectorless** RAG with PageIndex. The workflow illustrates the core idea:
> *Generating a hierarchical tree structure from a document, reasoning over that tree structure, and extracting relevant context, without relying on a vector database or top-k similarity search*.

While this notebook highlights a minimal workflow, the PageIndex framework is built to support **far more advanced** use cases. In upcoming tutorials, we will introduce:
* **Multi-Node Reasoning with Content Extraction** — Scale tree search to extract and select relevant content from multiple nodes.
* **Multi-Document Search** — Enable reasoning-based navigation across large document collections, extending beyond a single file.
* **Efficient Tree Search** — Improve tree search efficiency for long documents with a large number of nodes.
* **Expert Knowledge Integration and Preference Alignment** — Incorporate user preferences or expert insights by adding knowledge directly into the LLM tree search, without the need for fine-tuning.



## 🔎 Learn More About PageIndex
  <a href="https://vectify.ai">🏠 Homepage</a>&nbsp; • &nbsp;
  <a href="https://dash.pageindex.ai">🖥️ Dashboard</a>&nbsp; • &nbsp;
  <a href="https://docs.pageindex.ai/quickstart">📚 API Docs</a>&nbsp; • &nbsp;
  <a href="https://github.com/VectifyAI/PageIndex">📦 GitHub</a>&nbsp; • &nbsp;
  <a href="https://discord.com/invite/VuXuf29EUj">💬 Discord</a>&nbsp; • &nbsp;
  <a href="https://ii2abc2jejf.typeform.com/to/tK3AXl8T">✉️ Contact</a>

<br>

© 2025 [Vectify AI](https://vectify.ai)